In [1]:
from pyspark.sql import SparkSession

# Initialize a Spark session
spark = SparkSession.builder \
    .appName("Pandas with PySpark") \
    .getOrCreate()

df = spark.read.csv("s3://project.small.data.aml/small_data/HI-Medium_Trans.csv", header=True, inferSchema=True)

df1 = spark.read.csv("s3://project.small.data.aml/small_data/LI-Medium_Trans.csv", header=True, inferSchema=True)



Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2,application_1724084376435_0003,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from pyspark.sql import SparkSession

# Initialize a Spark session
spark = SparkSession.builder \
    .appName("Pandas with PySpark") \
    .getOrCreate()



df = spark.read.csv("s3://project.small.data.aml/small_data/LI-Small_Trans.csv", header=True, inferSchema=True)

df1 = spark.read.csv("s3://project.small.data.aml/small_data/HI-Small_Trans.csv", header=True, inferSchema=True)


uniondf = df.union(df1)

uniondf.count()



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

12002394

In [3]:
uniondf.write.mode("overwrite").parquet("s3://newmlmain/newml321/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
uniondf=uniondf.coalesce(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
uniondf.show(1000)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+---------+---------+-------+---------+---------------+------------------+--------------+----------------+--------------+-------------+
|       Timestamp|From Bank| Account2|To Bank| Account4|Amount Received|Receiving Currency|   Amount Paid|Payment Currency|Payment Format|Is Laundering|
+----------------+---------+---------+-------+---------+---------------+------------------+--------------+----------------+--------------+-------------+
|2022/09/01 00:08|       11|8000ECA90|     11|8000ECA90|      3195403.0|         US Dollar|     3195403.0|       US Dollar|  Reinvestment|            0|
|2022/09/01 00:21|     3402|80021DAD0|   3402|80021DAD0|        1858.96|         US Dollar|       1858.96|       US Dollar|  Reinvestment|            0|
|2022/09/01 00:00|       11|8000ECA90|   1120|8006AA910|       592571.0|         US Dollar|      592571.0|       US Dollar|        Cheque|            0|
|2022/09/01 00:16|     3814|8006AD080|   3814|8006AD080|          12.32|         U

In [6]:
uniondf.describe()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[summary: string, Timestamp: string, From Bank: string, Account2: string, To Bank: string, Account4: string, Amount Received: string, Receiving Currency: string, Amount Paid: string, Payment Currency: string, Payment Format: string, Is Laundering: string]

In [7]:

# Summary
# Timestamp: Convert to TimestampType for time-based operations.
# Bank and Account Identifiers: Retain as StringType.
# Amount Columns: Convert to FloatType or DoubleType for precise calculations.
# Currency Columns: Retain as StringType (currency codes).
# Payment Format: Retain as StringType.
# Is Laundering: Convert to IntegerType for binary classification.
# This approach ensures that each column is in the appropriate format for further analysis or processing.



from pyspark.sql import SparkSession
from pyspark.sql.functions import to_timestamp, col
from pyspark.sql.types import FloatType, IntegerType, StringType



uniondf = uniondf.withColumn("Timestamp", to_timestamp(col("Timestamp"), "yyyy/MM/dd HH:mm"))



uniondf = uniondf.withColumn("Amount Received", col("Amount Received").cast(FloatType()))
uniondf = uniondf.withColumn("Receiving Currency", col("Receiving Currency").cast(StringType()))
uniondf = uniondf.withColumn("Amount Paid", col("Amount Paid").cast(FloatType()))


uniondf = uniondf.withColumn("Is Laundering", col("Is Laundering").cast(IntegerType()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
uniondf.describe()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[summary: string, From Bank: string, Account2: string, To Bank: string, Account4: string, Amount Received: string, Receiving Currency: string, Amount Paid: string, Payment Currency: string, Payment Format: string, Is Laundering: string]

In [9]:
uniondf.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Timestamp: timestamp (nullable = true)
 |-- From Bank: integer (nullable = true)
 |-- Account2: string (nullable = true)
 |-- To Bank: integer (nullable = true)
 |-- Account4: string (nullable = true)
 |-- Amount Received: float (nullable = true)
 |-- Receiving Currency: string (nullable = true)
 |-- Amount Paid: float (nullable = true)
 |-- Payment Currency: string (nullable = true)
 |-- Payment Format: string (nullable = true)
 |-- Is Laundering: integer (nullable = true)

In [10]:
uniondf = uniondf.withColumn("From Bank", col("From Bank").cast(StringType()))
uniondf = uniondf.withColumn("Account2", col("Account2").cast(StringType()))
uniondf = uniondf.withColumn("To Bank", col("To Bank").cast(StringType()))
uniondf = uniondf.withColumn("Account4", col("Account4").cast(StringType()))
uniondf = uniondf.withColumn("Receiving Currency", col("Receiving Currency").cast(StringType()))
uniondf = uniondf.withColumn("Payment Currency", col("Payment Currency").cast(StringType()))
uniondf = uniondf.withColumn("Payment Format", col("Payment Format").cast(StringType()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
from pyspark.ml.feature import StringIndexer,VectorAssembler,MinMaxScaler

uniondf.show()


unique_values_A = uniondf.select('Receiving Currency').distinct()

unique_values_A.show()

from pyspark.sql.functions import col, when

conversion_rates = {
    "Yen": 0.0068,  # Example rate for USD
    "Swiss Franc": 1.15,       # Example rate for Euro
    "Ruble": 0.011,
    "Saudi Riyal": 0.27,
    "Mexican Peso":0.053,
    "Canadian Dollar":0.73,
    "Brazil Real": 0.20,              # Example rate to US Dollar
    "Bitcoin": 20000,                 # Example rate to US Dollar (1 Bitcoin = 20,000 USD)
    "UK Pound": 1.35,                 # Example rate to US Dollar
    "Yuan": 0.14,                     # Example rate to US Dollar
    "Rupee": 0.012,                   # Example rate to US Dollar
    "Australian Dollar": 0.65,        # Example rate to US Dollar
    "Shekel": 0.29,
    "Euro": 1.1,# Example rate for GBP
}


uniondf = uniondf.withColumn(
    "Amount Received",
     when(col("Receiving Currency") == "Yen", col("Amount Received") * conversion_rates["Yen"])
    .when(col("Receiving Currency") == "Swiss Franc", col("Amount Received") * conversion_rates["Swiss Franc"])
    .when(col("Receiving Currency") == "Ruble", col("Amount Received") * conversion_rates["Ruble"])
    .when(col("Receiving Currency") == "Saudi Riyal", col("Amount Received") * conversion_rates["Saudi Riyal"])
    .when(col("Receiving Currency") == "Mexican Peso", col("Amount Received") * conversion_rates["Mexican Peso"])
    .when(col("Receiving Currency") == "Canadian Dollar", col("Amount Received") * conversion_rates["Canadian Dollar"])
    .when(col("Receiving Currency") == "Brazil Real", col("Amount Received") * conversion_rates["Brazil Real"])
    .when(col("Receiving Currency") == "Bitcoin", col("Amount Received") * conversion_rates["Bitcoin"])
    .when(col("Receiving Currency") == "UK Pound", col("Amount Received") * conversion_rates["UK Pound"])
    .when(col("Receiving Currency") == "Yuan", col("Amount Received") * conversion_rates["Yuan"])
    .when(col("Receiving Currency") == "Rupee", col("Amount Received") * conversion_rates["Rupee"])
    .when(col("Receiving Currency") == "Australian Dollar", col("Amount Received") * conversion_rates["Australian Dollar"])
    .when(col("Receiving Currency") == "Shekel", col("Amount Received") * conversion_rates["Shekel"])
    .when(col("Receiving Currency") == "Euro", col("Amount Received") * conversion_rates["Euro"])
    .otherwise(col("Amount Received"))  # No conversion if currency is not listed
)



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+---------+---------+-------+---------+---------------+------------------+-----------+----------------+--------------+-------------+
|          Timestamp|From Bank| Account2|To Bank| Account4|Amount Received|Receiving Currency|Amount Paid|Payment Currency|Payment Format|Is Laundering|
+-------------------+---------+---------+-------+---------+---------------+------------------+-----------+----------------+--------------+-------------+
|2022-09-01 00:08:00|       11|8000ECA90|     11|8000ECA90|      3195403.0|         US Dollar|  3195403.0|       US Dollar|  Reinvestment|            0|
|2022-09-01 00:21:00|     3402|80021DAD0|   3402|80021DAD0|        1858.96|         US Dollar|    1858.96|       US Dollar|  Reinvestment|            0|
|2022-09-01 00:00:00|       11|8000ECA90|   1120|8006AA910|       592571.0|         US Dollar|   592571.0|       US Dollar|        Cheque|            0|
|2022-09-01 00:16:00|     3814|8006AD080|   3814|8006AD080|          12.32|       

In [12]:
conversion_rates = {
    "Yen": 0.0068,  # Example rate for USD
    "Swiss Franc": 1.15,       # Example rate for Euro
    "Ruble": 0.011,
    "Saudi Riyal": 0.27,
    "Mexican Peso":0.053,
    "Canadian Dollar":0.73,
    "Brazil Real": 0.20,              # Example rate to US Dollar
    "Bitcoin": 20000,                 # Example rate to US Dollar (1 Bitcoin = 20,000 USD)
    "UK Pound": 1.35,                 # Example rate to US Dollar
    "Yuan": 0.14,                     # Example rate to US Dollar
    "Rupee": 0.012,                   # Example rate to US Dollar
    "Australian Dollar": 0.65,        # Example rate to US Dollar
    "Shekel": 0.29,
    "Euro": 1.1,# Example rate for GBP
}


uniondf = uniondf.withColumn(
    "Amount Paid",
     when(col("Receiving Currency") == "Yen", col("Amount Received") * conversion_rates["Yen"])
    .when(col("Receiving Currency") == "Swiss Franc", col("Amount Received") * conversion_rates["Swiss Franc"])
    .when(col("Receiving Currency") == "Ruble", col("Amount Received") * conversion_rates["Ruble"])
    .when(col("Receiving Currency") == "Saudi Riyal", col("Amount Received") * conversion_rates["Saudi Riyal"])
    .when(col("Receiving Currency") == "Mexican Peso", col("Amount Received") * conversion_rates["Mexican Peso"])
    .when(col("Receiving Currency") == "Canadian Dollar", col("Amount Received") * conversion_rates["Canadian Dollar"])
    .when(col("Receiving Currency") == "Brazil Real", col("Amount Received") * conversion_rates["Brazil Real"])
    .when(col("Receiving Currency") == "Bitcoin", col("Amount Received") * conversion_rates["Bitcoin"])
    .when(col("Receiving Currency") == "UK Pound", col("Amount Received") * conversion_rates["UK Pound"])
    .when(col("Receiving Currency") == "Yuan", col("Amount Received") * conversion_rates["Yuan"])
    .when(col("Receiving Currency") == "Rupee", col("Amount Received") * conversion_rates["Rupee"])
    .when(col("Receiving Currency") == "Australian Dollar", col("Amount Received") * conversion_rates["Australian Dollar"])
    .when(col("Receiving Currency") == "Shekel", col("Amount Received") * conversion_rates["Shekel"])
    .when(col("Receiving Currency") == "Euro", col("Amount Received") * conversion_rates["Euro"])
    .otherwise(col("Amount Paid"))  # No conversion if currency is not listed
)



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
df_dropped = uniondf.drop("Receiving Currency", "Payment Currency")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
from pyspark.sql import functions as F

# Calculate the sum of the 'Amount Paid' column
sum_value = df_dropped.agg(F.sum("Amount Paid").alias("Total Sum")).collect()[0]["Total Sum"]

# Print the sum
print(f"Sum of 'Amount Paid': {sum_value}")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Sum of 'Amount Paid': 4784385723540295.0

In [15]:
sum_value = df_dropped.agg(F.sum("Amount Received").alias("Total Sum")).collect()[0]["Total Sum"]

# Print the sum
print(f"Sum of 'Amount Paid': {sum_value}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Sum of 'Amount Paid': 4135962820317.391

In [76]:
from pyspark.sql.functions import round
df_rounded = df_dropped.withColumn(
    "Amount Received", round(col("Amount Received"), 2)
)

df_rounded=df_rounded.withColumn(
    "Amount Paid", round(col("Amount Paid"), 2)
)

df_rounded.show()

df_rounded = df_rounded.coalesce(1)

df_rounded.count()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+---------+---------+-------+---------+---------------+-----------+--------------+-------------+
|          Timestamp|From Bank| Account2|To Bank| Account4|Amount Received|Amount Paid|Payment Format|Is Laundering|
+-------------------+---------+---------+-------+---------+---------------+-----------+--------------+-------------+
|2022-09-01 00:08:00|       11|8000ECA90|     11|8000ECA90|      3195403.0|  3195403.0|  Reinvestment|            0|
|2022-09-01 00:21:00|     3402|80021DAD0|   3402|80021DAD0|        1858.96|    1858.96|  Reinvestment|            0|
|2022-09-01 00:00:00|       11|8000ECA90|   1120|8006AA910|       592571.0|   592571.0|        Cheque|            0|
|2022-09-01 00:16:00|     3814|8006AD080|   3814|8006AD080|          12.32|      12.32|  Reinvestment|            0|
|2022-09-01 00:00:00|       20|8006AD530|     20|8006AD530|        2941.56|    2941.56|  Reinvestment|            0|
|2022-09-01 00:24:00|       12|8006ADD30|     12|8006ADD30|     

In [77]:
df_rounded=df_rounded.drop("Timestamp")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:

df_rounded.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+---------+-------+---------+---------------+-----------+--------------+-------------+
|From Bank| Account2|To Bank| Account4|Amount Received|Amount Paid|Payment Format|Is Laundering|
+---------+---------+-------+---------+---------------+-----------+--------------+-------------+
|       11|8000ECA90|     11|8000ECA90|      3195403.0|  3195403.0|  Reinvestment|            0|
|     3402|80021DAD0|   3402|80021DAD0|        1858.96|    1858.96|  Reinvestment|            0|
|       11|8000ECA90|   1120|8006AA910|       592571.0|   592571.0|        Cheque|            0|
|     3814|8006AD080|   3814|8006AD080|          12.32|      12.32|  Reinvestment|            0|
|       20|8006AD530|     20|8006AD530|        2941.56|    2941.56|  Reinvestment|            0|
|       12|8006ADD30|     12|8006ADD30|        6473.62|    6473.62|  Reinvestment|            0|
|       11|800059120|   1217|8006AD4E0|        60562.0|    60562.0|           ACH|            0|
|       11|8000ECA90|     11|8

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'df_filtered' is not defined
Traceback (most recent call last):
NameError: name 'df_filtered' is not defined



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'df_filtered' is not defined
Traceback (most recent call last):
NameError: name 'df_filtered' is not defined



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'df_filtered' is not defined
Traceback (most recent call last):
NameError: name 'df_filtered' is not defined



In [32]:
df_rounded.write.mode("overwrite").parquet("s3://newmlmain/withtimestamp/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Interrupted by user


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'df_rounded' is not defined
Traceback (most recent call last):
NameError: name 'df_rounded' is not defined



In [78]:
df_rounded.write.mode("overwrite").parquet("s3://mlpro11/mk111/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [71]:
df_rounded=spark.read.parquet("s3://mlpro11/twolakhsrows/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [79]:
filtered_df = df_rounded.filter(df_rounded["Is Laundering"] == 1)

filtered_df1 = df_rounded.filter(df_rounded["Is Laundering"] == 0)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [80]:
fraction = 0.2

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [81]:
sampled_df = filtered_df.sample(withReplacement=False, fraction=fraction, seed=42)

# Limit the result to 10,000 rows
limited_df = sampled_df.limit(10000)

limited_df.count()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1805

In [82]:
fraction = 0.1  # Adjust this fraction as needed

# Sample a fraction of the DataFrame
sampled_df1 = filtered_df1.sample(withReplacement=False, fraction=fraction, seed=42)

# Limit the result to 10,000 rows
limited_df1 = sampled_df1.limit(20000)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
limited_df1.count()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

200000

In [83]:
df_rounded=limited_df1.union(limited_df)

df_rounded.show()



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+---------+-------+---------+---------------+-----------+--------------+-------------+
|From Bank| Account2|To Bank| Account4|Amount Received|Amount Paid|Payment Format|Is Laundering|
+---------+---------+-------+---------+---------------+-----------+--------------+-------------+
|       11|8000ECA90|     11|8000ECA90|          22.97|      22.97|  Reinvestment|            0|
|      394|80093BDB0|    394|80093BDB0|       14903.78|   14903.78|  Reinvestment|            0|
|       20|800952940|     20|800952940|       21126.92|   21126.92|  Reinvestment|            0|
|    32050|800BA59F0|  32050|800BA59F0|         609.17|     609.17|  Reinvestment|            0|
|       12|800BA4BE0|     12|800BA4BE0|           14.7|       14.7|  Reinvestment|            0|
|     1231|800BBE260| 261455|817C3BC30|          189.6|      189.6|   Credit Card|            0|
|     1439|800BBF6C0|   1439|800BBF6C0|          11.98|      11.98|  Reinvestment|            0|
|    32081|800BC6D10|   1439|8

In [48]:
df_rounded.describe()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[summary: string, From Bank: string, Account2: string, To Bank: string, Account4: string, Amount Received: string, Amount Paid: string, Payment Format: string, Is Laundering: string]

In [40]:
df_rounded.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

201805

In [84]:
df_rounded.write.mode("overwrite").parquet("s3://mlpro11/twolakhsrows/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [85]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler
from pyspark.ml import Pipeline

indexers = [
    StringIndexer(inputCol="From Bank", outputCol="from_bank_index",handleInvalid="keep"),
    StringIndexer(inputCol="To Bank", outputCol="to_bank_index",handleInvalid="keep"),
    StringIndexer(inputCol="Payment Format", outputCol="type_index",handleInvalid="keep")
]

# OneHotEncoding the indexed columns
encoders = [
    OneHotEncoder(inputCol="from_bank_index", outputCol="from_bank_encoded"),
    OneHotEncoder(inputCol="to_bank_index", outputCol="to_bank_encoded"),
    OneHotEncoder(inputCol="type_index", outputCol="type_encoded")
]

# Assembling all features into a single vector
assembler = VectorAssembler(
    inputCols=["from_bank_encoded", "to_bank_encoded", "type_encoded", "Amount Paid", "Amount Received"],
    outputCol="features"
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [86]:
scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withMean=True, withStd=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [87]:
pipeline = Pipeline(stages=indexers + encoders + [assembler, scaler])


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [88]:
data_spark_preprocessed = pipeline.fit(df_rounded).transform(df_rounded)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [89]:
data_spark_preprocessed.select("scaled_features").show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|     scaled_features|
+--------------------+
|[-0.1806316401868...|
|[-0.1806316401868...|
|[-0.1806316401868...|
|[-0.1806316401868...|
|[-0.1806316401868...|
+--------------------+
only showing top 5 rows

In [90]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [91]:
train_data, test_data = data_spark_preprocessed.randomSplit([0.7, 0.3], seed=42)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [94]:
# Show the size of the training and test sets
print(f"Training Data Count: {train_data.count()}")
print(f"Test Data Count: {test_data.count()}")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Training Data Count: 15328
Test Data Count: 6477

In [101]:
rf_classifier = RandomForestClassifier(labelCol="Is Laundering", featuresCol="scaled_features", numTrees=100)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [102]:
rf_model = rf_classifier.fit(train_data)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [104]:
predictions = rf_model.transform(test_data)

# Initialize an evaluator with metrics like accuracy, precision, recall, and F1-score
evaluator = MulticlassClassificationEvaluator(labelCol="Is Laundering", predictionCol="prediction", metricName="accuracy")

# Calculate accuracy
accuracy = evaluator.evaluate(predictions)
print(f"Test Set Accuracy: {accuracy}")

# Optionally, evaluate other metrics like precision, recall, and F1-score
precision_evaluator = MulticlassClassificationEvaluator(labelCol="Is Laundering", predictionCol="prediction", metricName="weightedPrecision")
recall_evaluator = MulticlassClassificationEvaluator(labelCol="Is Laundering", predictionCol="prediction", metricName="weightedRecall")
f1_evaluator = MulticlassClassificationEvaluator(labelCol="Is Laundering", predictionCol="prediction", metricName="f1")

precision = precision_evaluator.evaluate(predictions)
recall = recall_evaluator.evaluate(predictions)
f1_score = f1_evaluator.evaluate(predictions)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Test Set Accuracy: 0.9161648911533117

In [105]:
print(f"Test Set Precision: {precision}")
print(f"Test Set Recall: {recall}")
print(f"Test Set F1-Score: {f1_score}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Test Set Precision: 0.8393581077819594
Test Set Recall: 0.9161648911533117
Test Set F1-Score: 0.8760812930632103

In [ ]:
# Create a parameter grid for hyperparameter tuning
paramGrid = ParamGridBuilder() \
    .addGrid(rf_classifier.numTrees, [50, 100, 200]) \
    .addGrid(rf_classifier.maxDepth, [5, 10, 15]) \
    .build()

# Initialize CrossValidator
crossval = CrossValidator(estimator=rf_classifier,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=5)

# Perform cross-validation
cv_model = crossval.fit(train_data)

# Evaluate the best model
cv_predictions = cv_model.transform(test_data)
cv_accuracy = evaluator.evaluate(cv_predictions)
print(f"Cross-Validated Test Set Accuracy: {cv_accuracy}")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [92]:
from pyspark.ml.classification import LogisticRegression

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [93]:
lr = LogisticRegression(featuresCol="scaled_features", labelCol="Is Laundering")  # Make sure you have a 'label' column



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [95]:
lr_model = lr.fit(train_data)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [97]:
predictions = lr_model.transform(test_data)

# Show some predictions
predictions.select("features", "IS Laundering", "prediction").show()



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------+----------+
|            features|IS Laundering|prediction|
+--------------------+-------------+----------+
|(6347,[13,3308,63...|            0|       0.0|
|(6347,[13,3308,63...|            0|       0.0|
|(6347,[13,3357,63...|            0|       0.0|
|(6347,[13,3311,63...|            0|       0.0|
|(6347,[13,3367,63...|            0|       0.0|
|(6347,[13,3400,63...|            0|       0.0|
|(6347,[13,3308,63...|            0|       0.0|
|(6347,[13,4504,63...|            0|       0.0|
|(6347,[13,3308,63...|            0|       0.0|
|(6347,[13,3308,63...|            0|       0.0|
|(6347,[13,3308,63...|            0|       0.0|
|(6347,[13,3310,63...|            0|       1.0|
|(6347,[13,3308,63...|            0|       0.0|
|(6347,[13,3308,63...|            0|       0.0|
|(6347,[13,3308,63...|            0|       0.0|
|(6347,[13,3308,63...|            0|       0.0|
|(6347,[13,3620,63...|            0|       0.0|
|(6347,[13,3706,63...|            0|    

In [98]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
binary_evaluator = BinaryClassificationEvaluator(labelCol="Is Laundering", metricName="areaUnderROC")
roc_auc = binary_evaluator.evaluate(predictions)
print(f"Area Under ROC Curve (AUC): {roc_auc:.2f}")

# Binary Classification Evaluator for Accuracy
accuracy_evaluator = MulticlassClassificationEvaluator(
    labelCol="Is Laundering", 
    predictionCol="prediction", 
    metricName="accuracy"
)



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Area Under ROC Curve (AUC): 0.96

In [99]:
from pyspark.sql.functions import col

# Compute confusion matrix
tp = predictions.filter((col("Is Laundering") == 1) & (col("prediction") == 1)).count()
tn = predictions.filter((col("Is Laundering") == 0) & (col("prediction") == 0)).count()
fp = predictions.filter((col("Is Laundering") == 0) & (col("prediction") == 1)).count()
fn = predictions.filter((col("Is Laundering") == 1) & (col("prediction") == 0)).count()

print(f"True Positives: {tp}")
print(f"True Negatives: {tn}")
print(f"False Positives: {fp}")
print(f"False Negatives: {fn}")



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

True Positives: 434
True Negatives: 5773
False Positives: 161
False Negatives: 109

In [100]:
accuracy = accuracy_evaluator.evaluate(predictions)
print(f"Accuracy: {accuracy:.2f}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Accuracy: 0.96